# Moov AI - Data science test 

Objective of this notebook is to answer Question 2

In [140]:
# Import des données et des librairies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from IPython.display import display
import plotly.express as px
import plotly.graph_objects as go

# Import xgboost and sklearn
from xgboost import XGBClassifier, XGBRegressor
from sklearn.model_selection import train_test_split

In [141]:
# Import csv
df = pd.read_csv(r'/Users/philippebeliveau/Desktop/Notebook/Moov AI/stores_sales_forecasting.csv', encoding='ISO-8859-1')

# Transform 'Order Date' and 'Ship Date' to datetime
df['Order Date'] = pd.to_datetime(df['Order Date'])
df['Ship Date'] = pd.to_datetime(df['Ship Date']) 

# Transform the postal code in a categorical variable
df['Postal Code'] = df['Postal Code'].astype('str')
df['Row ID'] = df['Row ID'].astype('str')

print(f"Shape of the dataset: {df.shape}")
display(df.head(5).style.set_sticky().set_properties(**{'overflow-x': 'auto'}))

# show me the type
df.dtypes

Shape of the dataset: (2121, 21)


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08 00:00:00,2016-11-11 00:00:00,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.960000,2,0.000000,41.913600
1,2,CA-2016-152156,2016-11-08 00:00:00,2016-11-11 00:00:00,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back",731.940000,3,0.000000,219.582000
2,4,US-2015-108966,2015-10-11 00:00:00,2015-10-18 00:00:00,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.577500,5,0.450000,-383.031000
3,6,CA-2014-115812,2014-06-09 00:00:00,2014-06-14 00:00:00,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032,West,FUR-FU-10001487,Furniture,Furnishings,"Eldon Expressions Wood and Plastic Desk Accessories, Cherry Wood",48.860000,7,0.000000,14.169400
4,11,CA-2014-115812,2014-06-09 00:00:00,2014-06-14 00:00:00,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032,West,FUR-TA-10001539,Furniture,Tables,Chromcraft Rectangular Conference Tables,1706.184000,9,0.200000,85.309200


Row ID                   object
Order ID                 object
Order Date       datetime64[ns]
Ship Date        datetime64[ns]
Ship Mode                object
Customer ID              object
Customer Name            object
Segment                  object
Country                  object
City                     object
State                    object
Postal Code              object
Region                   object
Product ID               object
Category                 object
Sub-Category             object
Product Name             object
Sales                   float64
Quantity                  int64
Discount                float64
Profit                  float64
dtype: object

In [142]:
# Create a feature to compute the difference between the order date and the ship date
df['Delivery Delay'] = (df['Ship Date'] - df['Order Date']).dt.days

# Create a feature regarding the profit margin 
df['Profit Margin'] = df['Profit'] / df['Sales']

Assigning rows at every possible date between Order_date min and max

In [143]:
def adjust_dataset_for_daily_entries(df, date_col):
    """Ensures that the dataset has a row for every single day, filling missing days with NaN values."""
    df[date_col] = pd.to_datetime(df[date_col])
    all_dates = pd.date_range(start=df[date_col].min(), end=df[date_col].max(), freq='D')
    
    # Ensure all columns are retained, filling missing values with NaN
    full_df = pd.DataFrame(all_dates, columns=[date_col])
    df = full_df.merge(df, on=date_col, how='left')
    
    return df

df = adjust_dataset_for_daily_entries(df, 'Order Date')

In [144]:
# Print the number of nan values for the sales column 
print(f"Number of NaN values in the 'Sales' column: {df['Sales'].isna().sum()}")
print(f"Number of days in the dataset: {df['Order Date'].nunique()}")
# Calculate the number of days with missing sales values
missing_sales_days = df['Sales'].isna().sum()
# Calculate the percentage of days with missing sales values
missing_sales_percentage = missing_sales_days / df['Order Date'].nunique() * 100
print(f"Percentage of days with missing sales values: {missing_sales_percentage:.2f}%")

Number of NaN values in the 'Sales' column: 566
Number of days in the dataset: 1455
Percentage of days with missing sales values: 38.90%


This tells me that I should change the frequency of my time series

In [145]:
# Change my frequency from daily to weekly and give me the statistics of the na values
df_weekly = df.resample('W', on='Order Date').sum(numeric_only=True).reset_index()
print(f"Number of NaN values in the 'Sales' column: {df_weekly['Sales'].isna().sum()}")
print(f"Number of weeks in the dataset: {df_weekly['Sales'].shape[0]}")
# Calculate the number of weeks with missing sales values
missing_sales_weeks = df_weekly['Sales'].isna().sum()
# Calculate the percentage of weeks with missing sales values
missing_sales_percentage = missing_sales_weeks / df_weekly['Sales'].shape[0] * 100

print(f"Percentage of weeks with missing sales values: {missing_sales_percentage:.2f}%")

Number of NaN values in the 'Sales' column: 0
Number of weeks in the dataset: 208
Percentage of weeks with missing sales values: 0.00%


# Question #2 : Insights et interprétation
Identifie des insights qui, selon toi, peuvent contribuer à comprendre les variations des ventes.
- En tenant compte des parties prenantes visées par ta solution, comment interprètes-tu les résultats produits par ta solution ML? Comment cette solution ajoute-t-elle de la valeur pour ces parties prenantes?
- Selon toi, comment envisage-tu que les parties prenantes vont utiliser ta solution pour
tenter de comprendre comment augmenter les ventes?

Methods: 
- Use of unsupervised method to find insight and add value to the clients 
- They will understand what kind of features causes what
    - Need coefficient? 
    - How to interpret? 

**Differentiating Between "Understanding Sales Drivers" and "Forecasting"**
1. Sales Drivers (What Affects Sales?)
- Key Question: What store actions, pricing changes, and product mixes lead to increased or decreased sales?

## Unsupervised approach

In [ ]:
from sklearn.preprocessing import StandardScaler
import pandas as pd

def scale_features(df, feature_cols):
    """
    Scales numerical features using StandardScaler.
    
    Parameters:
        df (pd.DataFrame): The input dataframe.
        feature_cols (list): List of columns to scale.
        
    Returns:
        pd.DataFrame: DataFrame with scaled features.
        scaler: The fitted StandardScaler instance.
    """
    # Initialize the scaler
    scaler = StandardScaler()
    
    # Fit and transform the selected features
    scaled_values = scaler.fit_transform(df[feature_cols])
    
    # Create a DataFrame for the scaled features
    scaled_df = pd.DataFrame(scaled_values, columns=feature_cols, index=df.index)
    
    # Combine scaled features with the original DataFrame
    df_scaled = df.copy()
    for col in feature_cols:
        df_scaled[col] = scaled_df[col]
    
    return df_scaled, scaler

# Example usage
feature_columns = ['Sales', 'Discount', 'Delivery Delay']  # Replace with your actual columns
df_scaled, scaler = scale_features(df, feature_columns)


In [ ]:
# from sklearn.decomposition import PCA
# import matplotlib.pyplot as plt

# # Apply PCA
# pca = PCA(n_components=2)
# pca_result = pca.fit_transform(df_scaled)

# # Visualize
# plt.scatter(pca_result[:, 0], pca_result[:, 1], c=df['Sales'], cmap='viridis')
# plt.colorbar(label='Sales')
# plt.title('PCA Analysis of Sales Data')
# plt.show()

## Supervised approach

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import numpy as np

# Handle missing values: Fill or remove
df.fillna(0, inplace=True)  # Example: Replace missing values with 0 (adjust as needed)

# Feature engineering: Extract temporal features
df['Year'] = df['Order Date'].dt.year
df['Month'] = df['Order Date'].dt.month
df['Day_of_Week'] = df['Order Date'].dt.dayofweek

# Encode categorical variables
categorical_cols = ['Ship Mode', 'Segment', 'Region', 'Category', 'Sub-Category']
df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# Select features and target
# Select features and target
feature_cols = [col for col in df_encoded.columns if col not in ['Row ID', 'Order ID', 'Customer ID',  'Order Date', 'Ship Date', 
                                                                 'Customer Name', 'Country', 'City', 'Postal Code', 'Product ID', 'Product Name', 
                                                                 'State', 'Profit', 'Profit Margin','Quantity', 
                                                                 'Sales']]
X = df_encoded[feature_cols]
y = df_encoded['Sales']

X.columns


/var/folders/dj/q5s8lngd1msdbb76w1tn9x1r0000gn/T/ipykernel_66673/3853646899.py:7: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.



Index(['Discount', 'Delivery Delay', 'Year', 'Month', 'Day_of_Week',
       'Ship Mode_First Class', 'Ship Mode_Same Day', 'Ship Mode_Second Class',
       'Ship Mode_Standard Class', 'Segment_Consumer', 'Segment_Corporate',
       'Segment_Home Office', 'Region_Central', 'Region_East', 'Region_South',
       'Region_West', 'Category_Furniture', 'Sub-Category_Bookcases',
       'Sub-Category_Chairs', 'Sub-Category_Furnishings',
       'Sub-Category_Tables'],
      dtype='object')

In [ ]:
# Split based on time periods
train_data = df_encoded[df_encoded['Year'] < 2017]
test_data = df_encoded[df_encoded['Year'] == 2017]

X_train = train_data[feature_cols]
y_train = train_data['Sales']
X_test = test_data[feature_cols]
y_test = test_data['Sales']


In [ ]:
# Define and train the Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [ ]:
# Create interaction features
train_data['Discount_Region'] = train_data['Discount'] * train_data['Region_West']  # Example: Region West
train_data['Discount_Category'] = train_data['Discount'] * train_data['Region_Central']

# Test data 
test_data['Discount_Region'] = test_data['Discount'] * test_data['Region_West']  # Example: Region West
test_data['Discount_Category'] = test_data['Discount'] * test_data['Region_Central']


/var/folders/dj/q5s8lngd1msdbb76w1tn9x1r0000gn/T/ipykernel_66673/4073618067.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/dj/q5s8lngd1msdbb76w1tn9x1r0000gn/T/ipykernel_66673/4073618067.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/dj/q5s8lngd1msdbb76w1tn9x1r0000gn/T/ipykernel_66673/4073618067.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [ ]:
feature_cols += ['Discount_Region', 'Discount_Category']
X_train = train_data[feature_cols]
X_test = test_data[feature_cols]

# Re-train model
rf_model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [ ]:
# Get feature importance
importance = rf_model.feature_importances_
importance_df = pd.DataFrame({'Feature': feature_cols, 'Importance': importance})
importance_df = importance_df.sort_values(by='Importance', ascending=False)
print(importance_df)


                     Feature  Importance
3                      Month    0.141838
19  Sub-Category_Furnishings    0.117825
4                Day_of_Week    0.100569
1             Delivery Delay    0.080907
16        Category_Furniture    0.067793
0                   Discount    0.064498
20       Sub-Category_Tables    0.063248
18       Sub-Category_Chairs    0.055181
2                       Year    0.055033
17    Sub-Category_Bookcases    0.040149
9           Segment_Consumer    0.025876
14              Region_South    0.025175
13               Region_East    0.023880
10         Segment_Corporate    0.022286
7     Ship Mode_Second Class    0.020534
11       Segment_Home Office    0.017608
8   Ship Mode_Standard Class    0.016057
12            Region_Central    0.014769
5      Ship Mode_First Class    0.013753
15               Region_West    0.012696
21           Discount_Region    0.008197
22         Discount_Category    0.007530
6         Ship Mode_Same Day    0.004598


Segment-Specific Feature Importance

In [ ]:
regions = df_encoded['Region_West'].unique()
segment_importance = {}

for region in regions:
    # Filter data for the region
    region_data = df_encoded[df_encoded['Region_West'] == region]
    X_region = region_data[feature_cols]
    y_region = region_data['Sales']
    
    # Train model
    rf_region = RandomForestRegressor(n_estimators=100, random_state=42)
    rf_region.fit(X_region, y_region)
    
    # Get feature importance
    importance = rf_region.feature_importances_
    segment_importance[region] = pd.DataFrame({'Feature': feature_cols, 'Importance': importance}).sort_values(by='Importance', ascending=False)

# Display feature importance for each segment
for region, importance in segment_importance.items():
    print(f"Feature Importance for Region: {region}")
    print(importance)


KeyError: "['Discount_Region', 'Discount_Category'] not in index"

Partial Dependence Plots

In [ ]:
from sklearn.inspection import PartialDependenceDisplay

# Plot partial dependence for a single feature
PartialDependenceDisplay.from_estimator(rf_model, X_train, ['Discount'])
plt.title('Partial Dependence for Discount')
plt.show()

# Plot interaction partial dependence for two features
PartialDependenceDisplay.from_estimator(rf_model, X_train, [('Discount', 'Region_West')])
plt.title('Partial Dependence for Discount and Region')
plt.show()


ValueError: feature_names should not contain duplicates.

**Shipping delay**

In [ ]:
import plotly.express as px
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

def visualize_shipping_delay_effect(df):
    """
    Visualizes the effect of shipping delay on normalized Sales and Profit per region using Plotly.
    
    Parameters:
    df (pd.DataFrame): The dataset with 'Order Date', 'Ship Date', 'Region', 'Sales', and 'Profit'.
    """
    # Ensure dates are in datetime format
    df['Order Date'] = pd.to_datetime(df['Order Date'])
    df['Ship Date'] = pd.to_datetime(df['Ship Date'])
    
    # Calculate shipping delay
    df['Shipping Delay (Days)'] = (df['Ship Date'] - df['Order Date']).dt.days
    df = df[df['Shipping Delay (Days)'] >= 0]  # Remove negative values
    
    # Normalize sales and profit per region between -1 and 1
    def normalize_region(column):
        scaler = MinMaxScaler(feature_range=(-1, 1))
        return df.groupby('Region')[column].transform(lambda x: scaler.fit_transform(x.values.reshape(-1, 1)).flatten())
    
    df['Normalized Sales'] = normalize_region('Sales')
    df['Normalized Profit'] = normalize_region('Profit')
    
    # Group data by shipping delay and region
    delay_grouped = df.groupby(['Shipping Delay (Days)', 'Region'])[['Normalized Sales', 'Normalized Profit']].mean().reset_index()
    
    # Plot Sales effect
    fig_sales = px.line(delay_grouped, x='Shipping Delay (Days)', y='Normalized Sales', color='Region',
                         title='Effect of Shipping Delay on Normalized Sales per Region',
                         markers=True, labels={'Normalized Sales': 'Avg Normalized Sales'})
    
    # Plot Profit effect
    fig_profit = px.line(delay_grouped, x='Shipping Delay (Days)', y='Normalized Profit', color='Region',
                          title='Effect of Shipping Delay on Normalized Profit per Region',
                          markers=True, labels={'Normalized Profit': 'Avg Normalized Profit'})
    
    # Show plots
    fig_sales.show()
    fig_profit.show()

# Example usage
visualize_shipping_delay_effect(df)


/var/folders/dj/q5s8lngd1msdbb76w1tn9x1r0000gn/T/ipykernel_66673/2495879302.py:25: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/dj/q5s8lngd1msdbb76w1tn9x1r0000gn/T/ipykernel_66673/2495879302.py:26: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [170]:
import plotly.express as px
import pandas as pd

def visualize_shipping_delay_effect(df):
    """
    Visualizes the effect of shipping delay on Sales and Profit per region using Plotly.
    
    Parameters:
    df (pd.DataFrame): The dataset with 'Order Date', 'Ship Date', 'Region', 'Sales', and 'Profit'.
    """
    # Ensure dates are in datetime format
    df['Order Date'] = pd.to_datetime(df['Order Date'])
    df['Ship Date'] = pd.to_datetime(df['Ship Date'])
    
    # Calculate shipping delay
    df['Shipping Delay (Days)'] = (df['Ship Date'] - df['Order Date']).dt.days
    df = df[df['Shipping Delay (Days)'] >= 0]  # Remove negative values
    
    # Group data by shipping delay and region
    delay_grouped = df.groupby(['Shipping Delay (Days)', 'Region'])[['Sales', 'Profit']].mean().reset_index()
    
    # Plot Sales effect
    fig_sales = px.line(delay_grouped, x='Shipping Delay (Days)', y='Sales', color='Region',
                         title='Effect of Shipping Delay on Sales per Region',
                         markers=True, labels={'Sales': 'Average Sales'})
    
    # Plot Profit effect
    fig_profit = px.line(delay_grouped, x='Shipping Delay (Days)', y='Profit', color='Region',
                          title='Effect of Shipping Delay on Profit per Region',
                          markers=True, labels={'Profit': 'Average Profit'})
    
    # Show plots
    fig_sales.show()
    fig_profit.show()

# Example usage
visualize_shipping_delay_effect(df)


In [ ]:
import plotly.express as px
import pandas as pd

def visualize_shipping_delay_effect(df):
    """
    Visualizes the effect of shipping delay on Sales and Profit per region using Plotly.
    
    Parameters:
    df (pd.DataFrame): The dataset with 'Order Date', 'Ship Date', 'Region', 'Sales', and 'Profit'.
    """
    # Ensure dates are in datetime format
    df['Order Date'] = pd.to_datetime(df['Order Date'])
    df['Ship Date'] = pd.to_datetime(df['Ship Date'])
    
    # Calculate shipping delay
    df['Shipping Delay (Days)'] = (df['Ship Date'] - df['Order Date']).dt.days
    df = df[df['Shipping Delay (Days)'] >= 0]  # Remove negative values
    
    # Group data by shipping delay and region
    delay_grouped = df.groupby(['Shipping Delay (Days)', 'Sub-Category'])[['Sales', 'Profit']].mean().reset_index()
    
    # Plot Sales effect
    fig_sales = px.line(delay_grouped, x='Shipping Delay (Days)', y='Sales', color='Sub-Category',
                         title='Effect of Shipping Delay on Sales per Sub-Category',
                         markers=True, labels={'Sales': 'Average Sales'})
    
    # Plot Profit effect
    fig_profit = px.line(delay_grouped, x='Shipping Delay (Days)', y='Profit', color='Sub-Category',
                          title='Effect of Shipping Delay on Profit per Sub-Category',
                          markers=True, labels={'Profit': 'Average Profit'})
    
    # Show plots
    fig_sales.show()
    fig_profit.show()

# Example usage
visualize_shipping_delay_effect(df)


Shouldn't this analysis be done regarding the type of products that is being delay? 

**Discount's effect**

In [164]:
import plotly.express as px
import pandas as pd

def visualize_discount_effect(df):
    """
    Visualizes the effect of Discount on Sales and Profit per region using Plotly.
    
    Parameters:
    df (pd.DataFrame): The dataset with 'Discount', 'Region', 'Sales', and 'Profit'.
    """
    # Group data by discount and region
    discount_grouped = df.groupby(['Discount', 'Region'])[['Sales', 'Profit']].mean().reset_index()
    
    # Plot Sales effect
    fig_sales = px.line(discount_grouped, x='Discount', y='Sales', color='Region',
                         title='Effect of Discount on Sales per Region',
                         markers=True, labels={'Sales': 'Average Sales', 'Discount': 'Discount (%)'})
    
    # Plot Profit effect
    fig_profit = px.line(discount_grouped, x='Discount', y='Profit', color='Region',
                          title='Effect of Discount on Profit per Region',
                          markers=True, labels={'Profit': 'Average Profit', 'Discount': 'Discount (%)'})
    
    # Show plots
    fig_sales.show()
    fig_profit.show()

# Example usage
visualize_discount_effect(df)


Analysis of if a discount is useful

In [165]:
df['Is_Discount'] = df['Discount'] > 0

# Aggregate the sales before and after the discount

df['Sales_Before_7_Days'] = df['Sales'].shift(1).rolling(window=7).sum()
df['Sales_After_7_Days'] = df['Sales'].shift(-1).rolling(window=7).sum()

# Compute the impact of discount
df['Sales_Change_After_Discount'] = (df['Sales_After_7_Days'] - df['Sales_Before_7_Days']) / df['Sales_Before_7_Days']

import plotly.express as px

fig = px.line(df, x='Order Date', y='Sales_Change_After_Discount', color='Region',
              title='Sales Before, During, and After Discounts', 
              markers=True)
fig.show()

df[df['Region'] == '0']


,Order Date,Row ID,Order ID,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Sales,Quantity,Discount,Profit,Delivery Delay,Profit Margin,Is_Discount,Sales_Before_7_Days,Sales_After_7_Days,Sales_Change_After_Discount


In [163]:
# Filter on September 10, 2014 and region West 
df[(df['Order Date'] >= '2014-09-10') & (df['Order Date'] < '2014-09-25') & (df['Region'] == 'West')]


,Order Date,Row ID,Order ID,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Sales,Quantity,Discount,Profit,Delivery Delay,Profit Margin,Is_Discount,Sales_Before_7_Days,Sales_After_7_Days,Sales_Change_After_Discount
336,2014-09-10,7526,CA-2014-125731,2014-09-16,Standard Class,CL-12565,Clay Ludtke,Consumer,United States,Gresham,...,1487.040,5.0,0.20,148.7040,6.0,0.100000,True,864.024,1802.612,1.086299
338,2014-09-11,6312,CA-2014-112837,2014-09-16,Standard Class,LW-17125,Liz Willingham,Consumer,United States,Oxnard,...,127.950,3.0,0.00,21.7515,5.0,0.170000,False,1802.612,423.410,-0.765113
342,2014-09-13,8652,CA-2014-142769,2014-09-13,Same Day,RP-19390,Resi Plking,Consumer,United States,Seattle,...,14.190,3.0,0.00,5.5341,0.0,0.390000,False,4165.968,367.434,-0.911801
350,2014-09-17,3927,CA-2014-103940,2014-09-21,Standard Class,BN-11515,Bradley Nguyen,Consumer,United States,Seattle,...,35.340,2.0,0.00,13.4292,4.0,0.380000,False,NaN,NaN,NaN
353,2014-09-19,9041,CA-2014-138177,2014-09-24,Standard Class,ND-18460,Neil Ducich,Corporate,United States,Chandler,...,73.915,1.0,0.50,-45.8273,5.0,-0.620000,True,NaN,1578.718,NaN
363,2014-09-23,3352,US-2014-134733,2014-09-28,Standard Class,BM-11650,Brian Moss,Corporate,United States,San Diego,...,435.999,3.0,0.15,20.5176,5.0,0.047059,True,685.512,NaN,NaN
